# Importing Libraries

In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import warnings
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, f1_score, precision_score
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

warnings.filterwarnings('ignore')

# For NLP chatbot
from sklearn.metrics.pairwise import cosine_similarity
import random

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\farou\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\farou\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\farou\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Data Loading and Exploration


In [112]:
df = pd.read_csv('data/edited_skill_exchange_dataset.csv')


# Display basic information about the dataset


In [113]:
print("Dataset Shape:", df.shape)
print("\nData Types:")
print(df.dtypes)
print("\nSample Data:")
print(df.head())

Dataset Shape: (10000, 6)

Data Types:
user_id            int64
joinedDate        object
joinedCourses     object
skills            object
desired_skills    object
isVerified          bool
dtype: object

Sample Data:
   user_id  joinedDate                            joinedCourses  \
0        1  2022-08-28  Machine Learning, CSS, Excel, SQL, HTML   
1        2  2023-12-04  Data Science, Excel, Python, JavaScript   
2        3  2023-04-10          JavaScript, Python, Excel, Java   
3        4  2022-01-30              AI, Machine Learning, Excel   
4        5  2022-09-07                                   Python   

                               skills  \
0                           HTML, SQL   
1   HTML, CSS, JavaScript, Excel, SQL   
2                     HTML, CSS, Java   
3  HTML, Excel, SQL, Java, Blockchain   
4                 CSS, JavaScript, AI   

                                      desired_skills  isVerified  
0  CSS, Java, Machine Learning, Blockchain, Data ...       False  

## 2. Data Preparation and Cleaning


In [114]:
# Comprehensive Data Cleaning Pipeline
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from dateutil.parser import parse
import re

def clean_dataset(df):
    print("Starting comprehensive data cleaning process...")
    print(f"Initial shape: {df.shape}")

    # 1. Make a copy to avoid modifying the original dataframe
    df_clean = df.copy()

    # 2. Fix data types
    # Convert joinedDate to datetime with error handling
    df_clean['joinedDate'] = pd.to_datetime(df_clean['joinedDate'], errors='coerce')

    # Calculate membership duration (in days) from a fixed reference point
    reference_date = pd.Timestamp('2025-03-12')
    df_clean['membershipDuration'] = (reference_date - df_clean['joinedDate']).dt.days

    # 3. Standardize text columns and handle special characters
    def deep_clean_text(text):
        if not isinstance(text, str):
            return ""
        # Remove special characters except commas
        text = re.sub(r'[^\w\s,]', '', text)
        # Normalize whitespace around commas
        text = re.sub(r'\s*,\s*', ', ', text)
        # Split by comma, strip whitespace, remove empty items, and rejoin
        items = [item.strip() for item in text.split(',') if item.strip()]
        # Remove duplicates while preserving order
        seen = set()
        unique_items = [x for x in items if not (x in seen or seen.add(x))]
        return ', '.join(unique_items) if unique_items else ""

    # Apply advanced text cleaning to specified columns
    text_columns = ['joinedCourses', 'skills', 'desired_skills']
    for col in text_columns:
        df_clean[col] = df_clean[col].apply(deep_clean_text)

    # 4. Handle missing values
    # Count missing values before imputation
    missing_before = df_clean.isnull().sum()
    print("\nMissing values before imputation:")
    print(missing_before[missing_before > 0])

    # Fill missing text fields with empty strings
    for col in text_columns:
        df_clean[col] = df_clean[col].fillna("")

    # 5. Create derived features
    # Count features with consistent handling of empty strings
    df_clean['course_count'] = df_clean['joinedCourses'].apply(lambda x: len(x.split(',')) if x else 0)
    df_clean['skills_count'] = df_clean['skills'].apply(lambda x: len(x.split(',')) if x else 0)
    df_clean['desired_skills_count'] = df_clean['desired_skills'].apply(lambda x: len(x.split(',')) if x else 0)

    # Calculate skill overlap (common skills between a user's skills and desired skills)
    def calculate_overlap(row):
        if not isinstance(row['skills'], str) or not isinstance(row['desired_skills'], str):
            return 0
        skills_set = set([s.strip() for s in row['skills'].split(',') if s.strip()])
        desired_set = set([s.strip() for s in row['desired_skills'].split(',') if s.strip()])
        return len(skills_set.intersection(desired_set))

    df_clean['skill_overlap'] = df_clean.apply(calculate_overlap, axis=1)

    # 6. Handle boolean values consistently
    df_clean['isVerified'] = df_clean['isVerified'].astype(int)

    # 7. Handle numerical outliers using IQR method
    numerical_cols = ['membershipDuration', 'course_count', 'skills_count', 'desired_skills_count']

    for col in numerical_cols:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Count outliers
        outliers = ((df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)).sum()
        if outliers > 0:
            print(f"Found {outliers} outliers in {col}")

        # Cap outliers instead of removing them
        df_clean[col] = df_clean[col].clip(lower_bound, upper_bound)

    # 8. Add user activity score (weighted combination of features)
    df_clean['user_activity_score'] = (
        0.3 * df_clean['course_count'] / df_clean['course_count'].max() +
        0.3 * df_clean['skills_count'] / df_clean['skills_count'].max() +
        0.2 * df_clean['membershipDuration'] / df_clean['membershipDuration'].max() +
        0.2 * df_clean['isVerified']
    ).round(3)

    # 9. Consistency check - ensure user_id is unique
    if df_clean['user_id'].duplicated().any():
        print(f"Warning: Found {df_clean['user_id'].duplicated().sum()} duplicate user IDs")
        # Keep first occurrence of duplicated user_ids
        df_clean = df_clean.drop_duplicates(subset=['user_id'], keep='first')

    # 10. Final validation checks
    # Check for any remaining nulls
    remaining_nulls = df_clean.isnull().sum().sum()
    if remaining_nulls > 0:
        print(f"Warning: Still have {remaining_nulls} null values in the dataset")

    print(f"\nFinal shape after cleaning: {df_clean.shape}")
    print("Data cleaning completed successfully")

    return df_clean

# Apply the comprehensive cleaning function
df_cleaned = clean_dataset(df)

# Show a sample of the cleaned data
print("\nSample of cleaned data:")
print(df_cleaned.head())
df = df_cleaned

# Display summary statistics
print("\nSummary statistics:")
print(df_cleaned.describe())

# Print column information
print("\nColumn information after cleaning:")
for col in df_cleaned.columns:
    col_type = df_cleaned[col].dtype
    if col_type == 'object':
        n_unique = df_cleaned[col].nunique()
        print(f"{col}: {col_type} with {n_unique} unique values")
    else:
        print(f"{col}: {col_type}")

Starting comprehensive data cleaning process...
Initial shape: (10000, 6)

Missing values before imputation:
Series([], dtype: int64)
Found 26 outliers in skills_count
Found 35 outliers in desired_skills_count

Final shape after cleaning: (10000, 12)
Data cleaning completed successfully

Sample of cleaned data:
   user_id joinedDate                            joinedCourses  \
0        1 2022-08-28  Machine Learning, CSS, Excel, SQL, HTML   
1        2 2023-12-04  Data Science, Excel, Python, JavaScript   
2        3 2023-04-10          JavaScript, Python, Excel, Java   
3        4 2022-01-30              AI, Machine Learning, Excel   
4        5 2022-09-07                                   Python   

                               skills  \
0                           HTML, SQL   
1   HTML, CSS, JavaScript, Excel, SQL   
2                     HTML, CSS, Java   
3  HTML, Excel, SQL, Java, Blockchain   
4                 CSS, JavaScript, AI   

                                      desir


## 3. Data Understanding and Visualization

In [115]:
plt.figure(figsize=(15, 10))

# Distribution of course counts
plt.subplot(2, 2, 1)
sns.histplot(df['course_count'], kde=True)
plt.title('Distribution of Course Counts')
plt.xlabel('Number of Courses')
plt.ylabel('Frequency')

# Distribution of skill counts
plt.subplot(2, 2, 2)
sns.histplot(df['skills_count'], kde=True)
plt.title('Distribution of Skill Counts')
plt.xlabel('Number of Skills')
plt.ylabel('Frequency')

# Distribution of desired skill counts
plt.subplot(2, 2, 3)
sns.histplot(df['desired_skills_count'], kde=True)
plt.title('Distribution of Desired Skill Counts')
plt.xlabel('Number of Desired Skills')
plt.ylabel('Frequency')

# Membership duration distribution
plt.subplot(2, 2, 4)
sns.histplot(df['membershipDuration'], kde=True)
plt.title('Distribution of Membership Duration')
plt.xlabel('Days')
plt.ylabel('Frequency')

plt.tight_layout()
plt.savefig('distributions.png')
plt.close()

# Function to extract all unique skills/courses from a column

In [116]:
def extract_unique_items(df, column_name):
    all_items = []
    for items_str in df[column_name]:
        if isinstance(items_str, str):
            items = [item.strip() for item in items_str.split(',')]
            all_items.extend(items)
    return list(set(all_items))

# Get unique items


In [117]:
all_courses = extract_unique_items(df, 'joinedCourses')
all_skills = extract_unique_items(df, 'skills')
all_desired_skills = extract_unique_items(df, 'desired_skills')
print(f"\nTotal unique courses: {len(all_courses)}")
print(f"Total unique skills: {len(all_skills)}")
print(f"Total unique desired skills: {len(all_desired_skills)}")


Total unique courses: 12
Total unique skills: 13
Total unique desired skills: 13


# Top courses visualization

In [118]:
def plot_top_items(df, column_name, title, n=10):
    all_items = []
    for items_str in df[column_name]:
        if isinstance(items_str, str):
            items = [item.strip() for item in items_str.split(',')]
            all_items.extend(items)

    item_counts = pd.Series(all_items).value_counts().head(n)

    plt.figure(figsize=(12, 6))
    sns.barplot(x=item_counts.values, y=item_counts.index)
    plt.title(f'Top {n} {title}')
    plt.xlabel('Count')
    plt.tight_layout()
    plt.savefig(f'top_{column_name}.png')
    plt.close()

# Plot top items for each category
plot_top_items(df, 'joinedCourses', 'Courses')
plot_top_items(df, 'skills', 'Skills')
plot_top_items(df, 'desired_skills', 'Desired Skills')

## 4. Feature Engineering for Skill Matching


In [119]:
# Create binary features for skills and desired skills using CountVectorizer
def create_binary_features(df, column_name):
    vectorizer = CountVectorizer(tokenizer=lambda x: [item.strip() for item in x.split(',')])
    binary_features = vectorizer.fit_transform(df[column_name].fillna(''))
    binary_df = pd.DataFrame(binary_features.toarray(), columns=vectorizer.get_feature_names_out())
    return binary_df, vectorizer

# Create binary feature matrices
skills_binary, skills_vectorizer = create_binary_features(df, 'skills')
desired_skills_binary, desired_skills_vectorizer = create_binary_features(df, 'desired_skills')
courses_binary, courses_vectorizer = create_binary_features(df, 'joinedCourses')

# Combine features for clustering
combined_features = pd.concat([
    skills_binary,
    desired_skills_binary,
    courses_binary,
    df[['membershipDuration', 'course_count', 'skills_count', 'desired_skills_count', 'isVerified']].reset_index(drop=True)
], axis=1)


# Scale numerical features

In [120]:
scaler = StandardScaler()
numerical_features = ['membershipDuration', 'course_count', 'skills_count', 'desired_skills_count']
combined_features[numerical_features] = scaler.fit_transform(combined_features[numerical_features])

# Convert boolean to int
combined_features['isVerified'] = combined_features['isVerified'].astype(int)

print("\nFeature Engineering Complete")
print(f"Total features: {combined_features.shape[1]}")


Feature Engineering Complete
Total features: 43


## 5. Clustering with KMeans (KNN)


In [121]:
# Determine optimal number of clusters using Elbow Method
inertia = []
k_range = range(2, 8)  # For a small dataset, we'll test up to 7 clusters
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(combined_features)
    inertia.append(kmeans.inertia_)

# Plot Elbow Method
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.grid(True)
plt.savefig('elbow_method.png')
plt.close()

In [122]:
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(combined_features)

# Add cluster labels to the original dataframe
df['cluster'] = cluster_labels

# Analyze clusters
cluster_stats = df.groupby('cluster').agg({
    'membershipDuration': 'mean',
    'course_count': 'mean',
    'skills_count': 'mean',
    'desired_skills_count': 'mean',
    'isVerified': 'mean',
    'user_id': 'count'
}).rename(columns={'user_id': 'count'})

print("\nCluster Statistics:")
print(cluster_stats)

# Visualize clusters
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x='skills_count',
    y='desired_skills_count',
    hue='cluster',
    size='course_count',
    sizes=(50, 200),
    palette='viridis',
    data=df
)
plt.title('User Clusters by Skills and Desired Skills')
plt.xlabel('Number of Current Skills')
plt.ylabel('Number of Desired Skills')
plt.savefig('user_clusters.png')
plt.close()


Cluster Statistics:
         membershipDuration  course_count  skills_count  desired_skills_count  \
cluster                                                                         
0                798.812039      4.029075      4.471335              6.009419   
1                795.778088      4.151394      2.613546              3.712351   
2                799.176856      2.061499      2.435953              5.967977   
3                801.097391      1.820870      4.328696              3.661304   

         isVerified  count  
cluster                     
0          0.518428   2442  
1          0.497610   2510  
2          0.512009   2748  
3          0.494783   2300  


## 6. Skill Matching System using SVM

In [123]:
# Prepare data for SVM - we'll predict if a user would be interested in "Machine Learning" as an example
target_skill = "Machine Learning"

# Check if the target skill is in the user's desired skills
df['wants_' + target_skill.replace(' ', '_')] = df['desired_skills'].apply(
    lambda x: 1 if target_skill in x else 0
)

# Feature matrix for prediction
X_svm = combined_features
y_svm = df['wants_' + target_skill.replace(' ', '_')]

# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_svm, y_svm, test_size=0.3, random_state=42)

# Train SVM model
svm_model = SVC(kernel='linear', probability=True, random_state=42)
svm_model.fit(X_train, y_train)

# Evaluate model
y_pred = svm_model.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Not Interested', 'Interested'],
            yticklabels=['Not Interested', 'Interested'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix for {target_skill} Interest Prediction')
plt.savefig('confusion_matrix.png')
plt.close()

print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1556
           1       1.00      1.00      1.00      1444

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00      3000



# Skill Matching using Decision Tree



In [ ]:
# Skill Matching using Decision Tree
def skill_matching_comparison(df):
    # Create feature vectors for all users based on their skills
    # First, identify all unique skills in the dataset
    all_skills = set()
    for skills_str in df['skills'].dropna():
        if isinstance(skills_str, str):
            all_skills.update([skill.strip() for skill in skills_str.split(',')])

    # Create binary feature matrix for skills
    skill_features = pd.DataFrame(0, index=df.index, columns=list(all_skills))

    # Fill the feature matrix
    for idx, row in df.iterrows():
        if isinstance(row['skills'], str):
            user_skills = [skill.strip() for skill in row['skills'].split(',')]
            for skill in user_skills:
                if skill in skill_features.columns:
                    skill_features.loc[idx, skill] = 1

    # Add other relevant features
    X = pd.concat([
        skill_features,
        df[['membershipDuration', 'course_count', 'skills_count', 'isVerified']].reset_index(drop=True)
    ], axis=1)

    # Scale numerical features
    scaler = StandardScaler()
    numerical_cols = ['membershipDuration', 'course_count', 'skills_count']
    X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

    # Let's create multiple target variables for different skills to make the comparison more robust
    target_skills = ["Machine Learning", "Python", "JavaScript", "Data Science", "AI"]

    # Store results for each target skill
    skill_results = {}

    for target_skill in target_skills:
        print(f"\nEvaluating matching for skill: {target_skill}")

        # Create target variable - does the user have this skill
        y = df['skills'].apply(lambda x: 1 if isinstance(x, str) and target_skill in x else 0)

        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Define the models
        models = {
            'SVM': SVC(probability=True, random_state=42),
            'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
            'KNN': KNeighborsClassifier(n_neighbors=5)
        }

        # Dictionary to store results for this skill
        model_results = {}

        # Train and evaluate each model
        for model_name, model in models.items():
            # Train the model
            model.fit(X_train, y_train)

            # Make predictions
            y_pred = model.predict(X_test)

            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, zero_division=0)
            recall = recall_score(y_test, y_pred, zero_division=0)
            f1 = f1_score(y_test, y_pred, zero_division=0)

            # Cross-validation scores
            cv_f1 = cross_val_score(model, X, y, cv=5, scoring='f1')
            cv_precision = cross_val_score(model, X, y, cv=5, scoring='precision')
            cv_recall = cross_val_score(model, X, y, cv=5, scoring='recall')
            cv_accuracy = cross_val_score(model, X, y, cv=5, scoring='accuracy')

            # Store results
            model_results[model_name] = {
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'cv_accuracy_mean': np.mean(cv_accuracy),
                'cv_precision_mean': np.mean(cv_precision),
                'cv_recall_mean': np.mean(cv_recall),
                'cv_f1_mean': np.mean(cv_f1)
            }

            print(f"{model_name}: F1={f1:.3f}, Precision={precision:.3f}, Recall={recall:.3f}, Accuracy={accuracy:.3f}")

        skill_results[target_skill] = model_results

    # Create plots comparing the models for each metric
    metrics = ['accuracy', 'precision', 'recall', 'f1']
    metric_labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

    # Calculate average metrics across all skills
    avg_results = {}
    for model_name in models.keys():
        avg_results[model_name] = {
            metric: np.mean([skill_results[skill][model_name][metric] for skill in target_skills])
            for metric in metrics
        }

    # First plot: comparing models across all metrics
    plt.figure(figsize=(14, 8))

    x = np.arange(len(metrics))
    width = 0.25
    model_names = list(models.keys())

    for i, model_name in enumerate(model_names):
        values = [avg_results[model_name][metric] for metric in metrics]
        plt.bar(x + i*width, values, width, label=model_name)

    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('Model Performance Comparison Across All Skills')
    plt.xticks(x + width, metric_labels)
    plt.legend()
    plt.ylim(0, 1)
    plt.savefig('model_metrics_comparison.png')
    plt.close()

    # Second plot: comparing models for each skill on F1 score
    plt.figure(figsize=(15, 8))

    x = np.arange(len(target_skills))
    width = 0.25

    for i, model_name in enumerate(model_names):
        values = [skill_results[skill][model_name]['f1'] for skill in target_skills]
        plt.bar(x + i*width, values, width, label=model_name)

    plt.xlabel('Target Skills')
    plt.ylabel('F1 Score')
    plt.title('F1 Score Comparison by Skill')
    plt.xticks(x + width, target_skills, rotation=45)
    plt.legend()
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.savefig('f1_by_skill_comparison.png')
    plt.close()

    # Create a detailed model comparison dataframe
    comparison_data = []
    for model_name in model_names:
        for metric in metrics:
            comparison_data.append({
                'Model': model_name,
                'Metric': metric.capitalize(),
                'Score': avg_results[model_name][metric]
            })

    comparison_df = pd.DataFrame(comparison_data)

    # Print summary of best models
    print("\n=== Model Performance Summary ===")
    best_model_by_metric = {}
    for metric in metrics:
        best_model = max(model_names, key=lambda m: avg_results[m][metric])
        best_score = avg_results[best_model][metric]
        best_model_by_metric[metric] = (best_model, best_score)
        print(f"Best model for {metric.capitalize()}: {best_model} with score {best_score:.3f}")

    # Create detailed output
    results_overview = {
        'skill_results': skill_results,
        'avg_results': avg_results,
        'comparison_df': comparison_df,
        'best_model_by_metric': best_model_by_metric
    }

    # Return the final comparison and evaluation
    return results_overview

# Run comparison
comparison_results = skill_matching_comparison(df)

# Visualize the results as a table
print("\nDetailed Model Comparison:")
pivot_table = comparison_results['comparison_df'].pivot(index='Model', columns='Metric', values='Score')
print(pivot_table.round(3))

# Display a visual summary for easier interpretation
plt.figure(figsize=(12, 6))
sns.heatmap(pivot_table, annot=True, cmap='viridis', fmt=".3f", linewidths=.5)
plt.title('Model Performance Comparison')
plt.tight_layout()
plt.savefig('model_performance_heatmap.png')
plt.close()

# Find overall best model based on average of all metrics
overall_scores = {}
for model in comparison_results['avg_results'].keys():
    metrics = comparison_results['avg_results'][model]
    overall_scores[model] = sum(metrics.values()) / len(metrics)

best_overall_model = max(overall_scores, key=overall_scores.get)
print(f"\nOverall Best Model: {best_overall_model} with average score {overall_scores[best_overall_model]:.3f}")


Evaluating matching for skill: Machine Learning
SVM: F1=1.000, Precision=1.000, Recall=1.000, Accuracy=1.000
Decision Tree: F1=1.000, Precision=1.000, Recall=1.000, Accuracy=1.000
KNN: F1=0.786, Precision=1.000, Recall=0.648, Accuracy=0.965

Evaluating matching for skill: Python
SVM: F1=1.000, Precision=1.000, Recall=1.000, Accuracy=1.000
Decision Tree: F1=1.000, Precision=1.000, Recall=1.000, Accuracy=1.000
KNN: F1=0.991, Precision=0.999, Recall=0.984, Accuracy=0.995

Evaluating matching for skill: JavaScript
SVM: F1=1.000, Precision=1.000, Recall=1.000, Accuracy=1.000
Decision Tree: F1=1.000, Precision=1.000, Recall=1.000, Accuracy=1.000
KNN: F1=0.994, Precision=0.990, Recall=0.998, Accuracy=0.995

Evaluating matching for skill: Data Science


## 7. User Matching for Peer Learning


In [101]:
def find_learning_partners_enhanced(user_id, df, n_recommendations=3, algorithm='all'):
    # Get user data
    if user_id not in df['user_id'].values:
        return {"error": "User not found"}

    user = df[df['user_id'] == user_id].iloc[0]

    # Extract user's skill info
    user_desired_skills = [skill.strip() for skill in user['desired_skills'].split(',') if isinstance(user['desired_skills'], str)]
    user_current_skills = [skill.strip() for skill in user['skills'].split(',') if isinstance(user['skills'], str)]

    # Create feature matrix for all users
    # 1. First encode all unique skills in the dataset
    all_skills = set()
    for skills_str in df['skills'].dropna():
        if isinstance(skills_str, str):
            all_skills.update([skill.strip() for skill in skills_str.split(',')])

    skill_features = pd.DataFrame(0, index=df.index, columns=list(all_skills))

    # Fill the feature matrix
    for idx, row in df.iterrows():
        if isinstance(row['skills'], str):
            user_skills = [skill.strip() for skill in row['skills'].split(',')]
            for skill in user_skills:
                if skill in skill_features.columns:
                    skill_features.loc[idx, skill] = 1

    # Create a target variable - potential match for our user
    # A user is a potential match if they have at least one skill our user wants
    y_match = np.zeros(len(df))
    for idx, row in df.iterrows():
        if row['user_id'] != user_id and isinstance(row['skills'], str):
            other_skills = [skill.strip() for skill in row['skills'].split(',')]
            match_score = sum(1 for skill in user_desired_skills if skill in other_skills)
            y_match[idx] = 1 if match_score > 0 else 0

    # Combine with other user features
    X = pd.concat([
        skill_features,
        df[['membershipDuration', 'course_count', 'skills_count', 'isVerified']].reset_index(drop=True)
    ], axis=1)

    # Scale numerical features
    scaler = StandardScaler()
    numerical_cols = ['membershipDuration', 'course_count', 'skills_count']
    X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

    # Define and train the models
    models = {
        'SVM': SVC(probability=True, random_state=42),
        'KNN': KNeighborsClassifier(n_neighbors=5),
        'Decision Tree': DecisionTreeClassifier(random_state=42)
    }

    results = {}
    for model_name, model in models.items():
        # Train the model
        model.fit(X, y_match)

        # Get cross-validation scores
        cv_scores = cross_val_score(model, X, y_match, cv=5, scoring='f1')
        precision_scores = cross_val_score(model, X, y_match, cv=5, scoring='precision')

        results[model_name] = {
            'f1_score': np.mean(cv_scores),
            'precision': np.mean(precision_scores),
            'model': model
        }

    # Plot comparison results
    if algorithm == 'all':
        plt.figure(figsize=(10, 6))
        metrics = ['f1_score', 'precision']
        model_names = list(results.keys())

        X_axis = np.arange(len(model_names))
        width = 0.35

        plt.bar(X_axis - width/2, [results[model]['f1_score'] for model in model_names], width, label='F1 Score')
        plt.bar(X_axis + width/2, [results[model]['precision'] for model in model_names], width, label='Precision')

        plt.xticks(X_axis, model_names)
        plt.xlabel('Models')
        plt.ylabel('Scores')
        plt.title('Model Performance Comparison')
        plt.legend()
        plt.ylim(0, 1)
        plt.savefig('model_comparison.png')
        plt.close()

    # Select the best model or use the specified one
    if algorithm == 'all':
        # Find model with best F1 score
        best_model = max(results, key=lambda x: results[x]['f1_score'])
        selected_model = results[best_model]['model']
        print(f"Selected best model: {best_model} with F1 Score: {results[best_model]['f1_score']:.3f}")
    else:
        selected_model = models[algorithm]

    # Use the selected model to find matches
    # Calculate match probabilities for all users
    match_probs = selected_model.predict_proba(X)[:, 1]

    # Create a list of potential matches
    potential_matches = []
    for idx, row in df.iterrows():
        if row['user_id'] == user_id:
            continue  # Skip the user themselves

        # Only consider users with prediction probability > 0.5
        if match_probs[idx] > 0.5:
            other_skills = [skill.strip() for skill in row['skills'].split(',') if isinstance(row['skills'], str)]
            other_desired_skills = [skill.strip() for skill in row['desired_skills'].split(',') if isinstance(row['desired_skills'], str)]

            # Calculate exact matching skills for output
            matching_skills = [skill for skill in user_desired_skills if skill in other_skills]
            can_learn_from_you = [skill for skill in other_desired_skills if skill in user_current_skills]

            if matching_skills:  # Only include if there's at least one matching skill
                potential_matches.append({
                    'user_id': row['user_id'],
                    'skills': row['skills'],
                    'matching_skills': matching_skills,
                    'can_learn_from_you': can_learn_from_you,
                    'match_score': match_probs[idx]
                })

    # Sort by match probability (higher is better)
    sorted_matches = sorted(potential_matches, key=lambda x: x['match_score'], reverse=True)

    # Return results
    return {
        'model_comparison': results,
        'recommendations': sorted_matches[:n_recommendations]
    }

In [102]:
# Get recommendations using all models and comparing them
results = find_learning_partners_enhanced(user_id=79, df=df, n_recommendations=4)

# Print model comparison
print("Model Performance Comparison:")
for model, metrics in results['model_comparison'].items():
    print(f"{model}: F1 Score = {metrics['f1_score']:.3f}, Precision = {metrics['precision']:.3f}")

# Print recommendations
print("\nRecommended Learning Partners:")
for i, match in enumerate(results['recommendations'], 1):
    print(f"\nMatch {i}:")
    print(f"User ID: {match['user_id']}")
    print(f"Match score: {match['match_score']:.3f}")
    print(f"Can teach you: {', '.join(match['matching_skills'])}")
    print(f"Can learn from you: {', '.join(match['can_learn_from_you'])}")

# Or use a specific algorithm
svm_results = find_learning_partners_enhanced(user_id=1, df=df, algorithm='SVM')
print("svm_results",svm_results)

Selected best model: SVM with F1 Score: 1.000
Model Performance Comparison:
SVM: F1 Score = 1.000, Precision = 1.000
KNN: F1 Score = 0.992, Precision = 0.995
Decision Tree: F1 Score = 1.000, Precision = 1.000

Recommended Learning Partners:

Match 1:
User ID: 3
Match score: 1.000
Can teach you: CSS, Java
Can learn from you: Nodejs

Match 2:
User ID: 8
Match score: 1.000
Can teach you: CSS, Data Science
Can learn from you: Nodejs

Match 3:
User ID: 11
Match score: 1.000
Can teach you: CSS, Excel
Can learn from you: 

Match 4:
User ID: 12
Match score: 1.000
Can teach you: CSS, Excel
Can learn from you: HTML
svm_results {'model_comparison': {'SVM': {'f1_score': np.float64(1.0), 'precision': np.float64(1.0), 'model': SVC(probability=True, random_state=42)}, 'KNN': {'f1_score': np.float64(0.9793536566197508), 'precision': np.float64(0.9976379450018629), 'model': KNeighborsClassifier()}, 'Decision Tree': {'f1_score': np.float64(1.0), 'precision': np.float64(1.0), 'model': DecisionTreeClassif

## 8. NLP Chatbot for Skill Guidance

In [103]:
# First, let's create a dataset of common questions and their responses for our skill guidance chatbot
skill_conversation_data = [
    {
        'question': 'What courses should I take to learn Machine Learning?',
        'answer': 'To learn Machine Learning, start with Python, Statistics, and Linear Algebra basics, then take courses on ML algorithms, neural networks, and practical ML projects.'
    },
    {
        'question': 'How do I learn Data Science?',
        'answer': 'To learn Data Science, I recommend courses in Python or R, statistics, data visualization, machine learning, and big data technologies. Start with the fundamentals and then work on real-world projects.'
    },
    {
        'question': 'What skills do I need for web development?',
        'answer': 'For web development, core skills include HTML, CSS, and JavaScript. For frontend, learn frameworks like React, Vue, or Angular. For backend, consider Node.js, Django, or Ruby on Rails, along with database skills.'
    },
    {
        'question': 'How long does it take to learn Python?',
        'answer': 'Learning Python basics takes 2-4 weeks, becoming proficient takes 3-6 months, and mastery requires ongoing practice. The timeline depends on your prior programming experience and study consistency.'
    },
    {
        'question': 'Which is better to learn first, Java or Python?',
        'answer': 'Python is often recommended for beginners due to its simpler syntax and readability. Java has a steeper learning curve but is valuable for enterprise applications. Choose based on your career goals.'
    },
    {
        'question': 'What should I learn after HTML and CSS?',
        'answer': 'After HTML and CSS, learn JavaScript to add interactivity to websites. Then consider a frontend framework like React, Vue, or Angular, and basic backend concepts for full-stack development.'
    },
    {
        'question': 'How to start learning AI?',
        'answer': 'Start learning AI with Python programming, statistics, and linear algebra. Then progress to machine learning fundamentals, neural networks, and specialized areas like NLP or computer vision.'
    },
    {
        'question': 'What are the best resources to learn SQL?',
        'answer': 'Great SQL learning resources include interactive platforms like SQLZoo and Mode Analytics, courses on Coursera and DataCamp, and practice through real database projects.'
    },
    {
        'question': 'How do I find a study partner?',
        'answer': 'You can find a study partner by using our matching system, joining relevant online communities, participating in forums related to your interests, or attending virtual meetups and hackathons.'
    },
    {
        'question': 'What skills are in demand right now?',
        'answer': 'Currently in-demand skills include Machine Learning, Data Science, Cloud Computing (AWS/Azure), DevOps, Full-Stack Development, Cybersecurity, and Blockchain development.'
    }
]


# Preprocess text function (tokenization, removing stopwords, lemmatization)

In [104]:
nltk.download('punkt_tab')
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters
    text = re.sub(r'\W', ' ', text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

# Preprocess the conversation data
processed_questions = [preprocess_text(item['question']) for item in skill_conversation_data]

# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_questions)


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\farou\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [105]:
# Function to get the most similar response
def get_chatbot_response(user_input, threshold=0.3):
    # Preprocess the user input
    processed_input = preprocess_text(user_input)

    # Vectorize the user input
    user_vector = tfidf_vectorizer.transform([processed_input])

    # Calculate similarity scores
    similarity_scores = cosine_similarity(user_vector, tfidf_matrix)[0]

    # Get the index of the most similar question
    max_similarity_index = np.argmax(similarity_scores)
    max_similarity = similarity_scores[max_similarity_index]

    # If similarity is above threshold, return the corresponding answer
    if max_similarity >= threshold:
        return skill_conversation_data[max_similarity_index]['answer']
    else:
        # Generate a generic response for questions not in our dataset
        return "I'm not sure about that specific topic. However, I can help you find courses or study partners for various skills. Could you tell me what skills you're interested in learning?"



## Testing the NLP

In [106]:
# Test the chatbot with sample questions
sample_questions = [
    "What should I learn to become a data scientist?",
    "I want to learn web development, what should I study?",
    "How can I find someone to study programming with?",
    "What programming language should I learn first?",
    "I'm interested in artificial intelligence"
]

print("\nChatbot Response Examples:")
for question in sample_questions:
    print(f"\nQ: {question}")
    print(f"A: {get_chatbot_response(question)}")

## 9. Building a complete skill guidance system

def skill_guidance_system(user_id, user_query, df):
    """
    Combined skill guidance system that leverages all components:
    1. Chatbot for general skill guidance
    2. User matching for peer learning
    3. Course recommendations based on desired skills
    """
    response = {}

    # Get chatbot response
    chatbot_answer = get_chatbot_response(user_query)
    response['guidance'] = chatbot_answer

    # If user_id is provided, get personalized recommendations
    if user_id and user_id in df['user_id'].values:
        # Get matching learning partners
        learning_partners = find_learning_partners_enhanced(user_id, df, n_recommendations=2)
        response['learning_partners'] = learning_partners

        # Get user's desired skills
        user_desired_skills = df[df['user_id'] == user_id]['desired_skills'].iloc[0].split(', ')

        # Recommend courses based on desired skills (simple frequency-based recommendation)
        recommended_courses = []
        for skill in user_desired_skills:
            skill = skill.strip()
            # Find users who have this skill
            users_with_skill = df[df['skills'].apply(lambda x: skill in x)]
            # Get courses they took
            if not users_with_skill.empty:
                courses = []
                for course_list in users_with_skill['joinedCourses']:
                    courses.extend([c.strip() for c in course_list.split(',')])

                # Get most common courses
                if courses:
                    course_counts = pd.Series(courses).value_counts()
                    top_courses = course_counts.head(2).index.tolist()
                    recommended_courses.append({
                        'skill': skill,
                        'recommended_courses': top_courses
                    })

        response['course_recommendations'] = recommended_courses

    return response



Chatbot Response Examples:

Q: What should I learn to become a data scientist?
A: To learn Data Science, I recommend courses in Python or R, statistics, data visualization, machine learning, and big data technologies. Start with the fundamentals and then work on real-world projects.

Q: I want to learn web development, what should I study?
A: For web development, core skills include HTML, CSS, and JavaScript. For frontend, learn frameworks like React, Vue, or Angular. For backend, consider Node.js, Django, or Ruby on Rails, along with database skills.

Q: How can I find someone to study programming with?
A: You can find a study partner by using our matching system, joining relevant online communities, participating in forums related to your interests, or attending virtual meetups and hackathons.

Q: What programming language should I learn first?
A: Python is often recommended for beginners due to its simpler syntax and readability. Java has a steeper learning curve but is valuable fo

## Whole System Testing

In [108]:
test_user_id = 1
test_query = "I want to learn mACHINE LEARNING AND I DONT KNOW WHERE TO START"

guidance_result = skill_guidance_system(test_user_id, test_query, df)

print("\nComplete Skill Guidance System Output:")
print("\nChatbot Guidance:")
print(guidance_result['guidance'])

print("\nRecommended Learning Partners:")
for partner in guidance_result['learning_partners']:
    print(partner)

print("\nCourse Recommendations:")
for rec in guidance_result['course_recommendations']:
    print(f"For {rec['skill']}: {', '.join(rec['recommended_courses'])}")


Selected best model: SVM with F1 Score: 1.000

Complete Skill Guidance System Output:

Chatbot Guidance:
Start learning AI with Python programming, statistics, and linear algebra. Then progress to machine learning fundamentals, neural networks, and specialized areas like NLP or computer vision.

Recommended Learning Partners:
model_comparison
recommendations

Course Recommendations:
For CSS: Excel, HTML
For Java: CSS, Data Science
For Machine Learning: AI, Java
For Blockchain: Data Science, HTML
For Data Science: CSS, AI


# Chatbot with Enhanced Skill Matching without NLP

In [110]:
def enhanced_skill_chatbot(df):
    # Create training data for intent classification
    intents = [
        {
            'tag': 'course_recommendation',
            'patterns': [
                "What courses should I take to learn Machine Learning?",
                "How do I learn Data Science?",
                "Recommend courses for web development",
                "What should I study for AI?",
                "Best courses for Python",
                "Suggest courses for JavaScript",
                "How to learn blockchain",
                "Courses for SQL",
                "What should I learn for data analysis",
                "Recommend learning path for frontend"
            ],
            'responses': [
                "Based on your interests, I recommend starting with {}, then moving to {}.",
                "To learn {}, I suggest these courses: {}.",
                "The best learning path for {} includes: {}."
            ]
        },
        {
            'tag': 'skill_duration',
            'patterns': [
                "How long does it take to learn Python?",
                "Time needed to become a web developer",
                "How much time to learn JavaScript?",
                "How long until I can work as a data scientist?",
                "Months required to master machine learning",
                "Time investment for learning SQL",
                "How long to learn React",
                "Time needed for Java proficiency",
                "How long does it take to learn AI",
                "Duration to become skilled in blockchain"
            ],
            'responses': [
                "Learning {} typically takes {} months of consistent practice.",
                "Most people become proficient in {} after {} months of study.",
                "Expect to spend about {} months to master {}."
            ]
        },
        {
            'tag': 'find_partners',
            'patterns': [
                "How can I find a study partner?",
                "Looking for someone to practice coding with",
                "Need a partner to learn JavaScript",
                "Find me someone to study with",
                "How to find peer learners",
                "Connect me with study partners",
                "Study group for Python",
                "Partner for pair programming",
                "Need help finding learning buddies",
                "Match me with a study partner"
            ],
            'responses': [
                "I can help you find learning partners with similar interests.",
                "Let me match you with peers who are looking to study {}.",
                "Our matching algorithm can find you ideal study partners based on your interests."
            ]
        },
        {
            'tag': 'skill_comparison',
            'patterns': [
                "Which is better to learn first, Java or Python?",
                "React vs Angular, which should I learn?",
                "Compare data science and machine learning",
                "Python or R for data analysis?",
                "JavaScript or TypeScript for web development",
                "AWS vs Azure for cloud computing",
                "Flutter vs React Native",
                "Compare SQL and NoSQL",
                "Django vs Flask for Python web development",
                "TensorFlow or PyTorch for machine learning"
            ],
            'responses': [
                "When comparing {} and {}, consider your goals: {}.",
                "Both {} and {} are valuable, but {} might be better for beginners.",
                "For your needs, {} might be more suitable than {} because {}."
            ]
        },
        {
            'tag': 'trending_skills',
            'patterns': [
                "What skills are in demand right now?",
                "Popular programming languages in 2023",
                "Trending tech skills",
                "What should I learn to get a job?",
                "Most valuable skills in tech",
                "Current tech trends",
                "What are employers looking for",
                "Highest paid skills in tech",
                "What's hot in programming right now",
                "Future-proof skills to learn"
            ],
            'responses': [
                "Currently trending skills include: {}.",
                "Employers are actively seeking candidates with: {}.",
                "For maximum employability, consider learning: {}."
            ]
        }
    ]

    # Prepare training data
    X = []
    y = []

    for intent in intents:
        for pattern in intent['patterns']:
            X.append(pattern)
            y.append(intent['tag'])

    # Split data for training and evaluation
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Create pipeline with TF-IDF and classifiers
    classifiers = {
        'SVM': Pipeline([
            ('tfidf', TfidfVectorizer(stop_words='english')),
            ('clf', SVC(kernel='linear', probability=True, random_state=42))
        ]),
        'Decision Tree': Pipeline([
            ('tfidf', TfidfVectorizer(stop_words='english')),
            ('clf', DecisionTreeClassifier(random_state=42))
        ]),
        'KNN': Pipeline([
            ('tfidf', TfidfVectorizer(stop_words='english')),
            ('clf', KNeighborsClassifier(n_neighbors=5))
        ])
    }

    # Dictionary to store results
    results = {}

    # Train and evaluate each classifier
    for name, classifier in classifiers.items():
        # Cross-validation scores
        cv_accuracy = cross_val_score(classifier, X, y, cv=5, scoring='accuracy')
        cv_f1 = cross_val_score(classifier, X, y, cv=5, scoring='f1_macro')
        cv_precision = cross_val_score(classifier, X, y, cv=5, scoring='precision_macro')
        cv_recall = cross_val_score(classifier, X, y, cv=5, scoring='recall_macro')

        # Train on training set
        classifier.fit(X_train, y_train)

        # Test on test set
        y_pred = classifier.predict(X_test)

        # Store results
        results[name] = {
            'cv_accuracy_mean': np.mean(cv_accuracy),
            'cv_accuracy_std': np.std(cv_accuracy),
            'cv_f1_mean': np.mean(cv_f1),
            'cv_f1_std': np.std(cv_f1),
            'cv_precision_mean': np.mean(cv_precision),
            'cv_precision_std': np.std(cv_precision),
            'cv_recall': np.mean(cv_recall),
            'test_f1': f1_score(y_test, y_pred, average='macro'),
            'test_precision': precision_score(y_test, y_pred, average='macro'),
            'classifier': classifier,
            'classification_report': classification_report(y_test, y_pred)
        }

    # Plot comparison results
    plt.figure(figsize=(12, 8))

    metrics = ['cv_accuracy_mean', 'cv_f1_mean', 'cv_precision_mean', 'test_f1', 'test_precision']
    metric_labels = ['Accuracy', 'F1 Score', 'Precision', 'Test F1', 'Test Precision']
    model_names = list(results.keys())

    x = np.arange(len(model_names))
    width = 0.25

    for i, metric in enumerate(metrics):
        values = [results[model][metric] for model in model_names]
        plt.bar(x + i*width, values, width, label=metric_labels[i])

    plt.xlabel('Models')
    plt.ylabel('Score')
    plt.title('Classifier Performance Comparison')
    plt.xticks(x + width, model_names)
    plt.legend()
    plt.ylim(0, 1)
    plt.savefig('chatbot_classifier_comparison.png')
    plt.close()

    # Find best performing model based on F1 score
    best_model = max(results, key=lambda x: results[x]['cv_f1_mean'])

    # Define response function using the best classifier
    def get_response(user_input):
        # Predict intent
        intent_pred = results[best_model]['classifier'].predict([user_input])[0]
        intent_probs = results[best_model]['classifier'].predict_proba([user_input])[0]
        max_prob = max(intent_probs)

        # Find matching intent
        for intent in intents:
            if intent['tag'] == intent_pred:
                # Select a random response template
                response_template = np.random.choice(intent['responses'])

                # Fill in the template with context-specific information
                if intent['tag'] == 'course_recommendation':
                    # Extract skills from user input using keyword matching
                    skills = []
                    for skill in ["Python", "JavaScript", "Data Science", "Machine Learning",
                                 "SQL", "AI", "Web Development", "Java", "React", "Node.js"]:
                        if skill.lower() in user_input.lower():
                            skills.append(skill)

                    if not skills:
                        skills = ["programming", "tech skills"]

                    # Get top courses for those skills from our dataset
                    related_courses = []
                    for skill in skills[:2]:  # Use at most 2 skills
                        # Find users with this skill
                        users_with_skill = df[df['skills'].apply(lambda x: isinstance(x, str) and skill in x)]
                        if not users_with_skill.empty:
                            # Get courses they took
                            courses = []
                            for course_list in users_with_skill['joinedCourses']:
                                if isinstance(course_list, str):
                                    courses.extend([c.strip() for c in course_list.split(',')])

                            if courses:
                                # Get most frequent course
                                course_counts = pd.Series(courses).value_counts()
                                top_course = course_counts.index[0] if not course_counts.empty else "introductory courses"
                                related_courses.append(top_course)

                    if not related_courses:
                        related_courses = ["beginner tutorials", "online courses"]

                    return response_template.format(skills[0], ", ".join(related_courses))

                elif intent['tag'] == 'skill_duration':
                    # Extract skills from user input
                    skills = []
                    for skill in ["Python", "JavaScript", "Data Science", "Machine Learning",
                                 "SQL", "AI", "Web Development", "Java", "React", "Node.js"]:
                        if skill.lower() in user_input.lower():
                            skills.append(skill)

                    if not skills:
                        skill = "this technology"
                    else:
                        skill = skills[0]

                    # Estimate time based on complexity
                    complexity = {
                        "Python": "2-3", "JavaScript": "3-4", "HTML": "1-2", "CSS": "2-3",
                        "Java": "4-6", "SQL": "1-3", "Machine Learning": "6-12",
                        "Data Science": "6-12", "AI": "9-18", "Web Development": "4-8"
                    }

                    time = complexity.get(skill, "3-6")
                    return response_template.format(skill, time)

                # Add more intent-specific logic as needed

                # For intents without specific handling, just return a response
                return np.random.choice(intent['responses'])

        # Fallback response if no intent matched (shouldn't happen)
        return "I'm not sure I understand. Could you rephrase your question?"

    # Create a comprehensive response
    print("\n----- Chatbot Model Evaluation -----")
    print(f"Best performing model: {best_model}")
    print("\nModel Comparison:")
    for name, metrics in results.items():
        print(f"\n{name}:")
        print(f"  Cross-validation Accuracy: {metrics['cv_accuracy_mean']:.3f} (±{metrics['cv_accuracy_std']:.3f})")
        print(f"  Cross-validation F1 Score: {metrics['cv_f1_mean']:.3f} (±{metrics['cv_f1_std']:.3f})")
        print(f"  Cross-validation Precision: {metrics['cv_precision_mean']:.3f} (±{metrics['cv_precision_std']:.3f})")
        print(f"  Test F1 Score: {metrics['test_f1']:.3f}")
        print(f"  Test Precision: {metrics['test_precision']:.3f}")

    # Return both the evaluation results and the response function
    return {
        'evaluation': results,
        'get_response': get_response,
        'best_model': best_model
    }

# Test the enhanced chatbot
chatbot = enhanced_skill_chatbot(df)

# Test some questions
test_questions = [
    "What courses should I take to learn Machine Learning?",
    "How long does it take to learn Python?",
    "Can you help me find a study partner for JavaScript?",
    "Should I learn React or Angular first?",
    "What are the most in-demand skills right now?"
]

print("\n----- Chatbot Response Examples -----")
for question in test_questions:
    print(f"\nQ: {question}")
    print(f"A: {chatbot['get_response'](question)}")


----- Chatbot Model Evaluation -----
Best performing model: SVM

Model Comparison:

SVM:
  Cross-validation Accuracy: 0.680 (±0.040)
  Cross-validation F1 Score: 0.671 (±0.056)
  Cross-validation Precision: 0.710 (±0.065)
  Test F1 Score: 0.576
  Test Precision: 0.567

Decision Tree:
  Cross-validation Accuracy: 0.620 (±0.098)
  Cross-validation F1 Score: 0.598 (±0.116)
  Cross-validation Precision: 0.669 (±0.142)
  Test F1 Score: 0.377
  Test Precision: 0.378

KNN:
  Cross-validation Accuracy: 0.640 (±0.049)
  Cross-validation F1 Score: 0.635 (±0.053)
  Cross-validation Precision: 0.727 (±0.079)
  Test F1 Score: 0.700
  Test Precision: 0.800

----- Chatbot Response Examples -----

Q: What courses should I take to learn Machine Learning?
A: Based on your interests, I recommend starting with Machine Learning, then moving to AI.

Q: How long does it take to learn Python?
A: Most people become proficient in Python after 2-3 months of study.

Q: Can you help me find a study partner for Ja